In [1]:
import pandas as pd
import numpy as np
import hvplot.pandas
from pathlib import Path
from sklearn.decomposition import PCA

In [2]:
bitcoin_df = pd.read_csv(
    Path("Resources/augmento_btc.csv"), 
    index_col='date', 
    infer_datetime_format=True, 
    parse_dates=True
).dropna()

bitcoin_df.rename(columns={"listing_close": "btc_close"}, inplace=True)

display(bitcoin_df.head())
display(bitcoin_df.tail())

C:\Users\chris\AppData\Local\Temp\ipykernel_33892\864437904.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  bitcoin_df = pd.read_csv(


,btc_close,twitter_hacks,twitter_pessimistic_doubtful,twitter_banks,twitter_selling,twitter_market_manipulation,twitter_de_centralisation,twitter_angry,twitter_etf,twitter_leverage,...,reddit_buying,reddit_warning,reddit_annoyed_frustrated,reddit_price,reddit_use_case_applications,reddit_rumor,reddit_scam_fraud,reddit_airdrop,reddit_optimistic,reddit_negative
date,,,,,,,,,,,,,,,,,,,,,
2016-11-01 01:00:00,706.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,6.0,2.0,0.0,0.0,0.0,1.0,12.0
2016-11-01 02:00:00,709.99,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,2.0,0.0,0.0,4.0,2.0,0.0,0.0,0.0,1.0,8.0
2016-11-01 03:00:00,708.54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,2.0,7.0
2016-11-01 04:00:00,702.85,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,1.0,5.0
2016-11-01 05:00:00,708.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,5.0,3.0,0.0,0.0,0.0,1.0,7.0


,btc_close,twitter_hacks,twitter_pessimistic_doubtful,twitter_banks,twitter_selling,twitter_market_manipulation,twitter_de_centralisation,twitter_angry,twitter_etf,twitter_leverage,...,reddit_buying,reddit_warning,reddit_annoyed_frustrated,reddit_price,reddit_use_case_applications,reddit_rumor,reddit_scam_fraud,reddit_airdrop,reddit_optimistic,reddit_negative
date,,,,,,,,,,,,,,,,,,,,,
2023-12-21 06:00:00,43795.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,7.0,1.0,0.0,16.0,3.0,3.0,3.0,0.0,7.0,30.0
2023-12-21 07:00:00,43753.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,0.0,0.0,18.0,3.0,1.0,5.0,0.0,3.0,32.0
2023-12-21 08:00:00,43684.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,7.0,0.0,0.0,11.0,3.0,0.0,5.0,0.0,5.0,29.0
2023-12-21 09:00:00,43733.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,6.0,1.0,0.0,12.0,4.0,0.0,3.0,0.0,4.0,26.0
2023-12-21 10:00:00,44008.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,...,5.0,0.0,0.0,12.0,3.0,1.0,2.0,0.0,4.0,21.0


In [3]:
pca = PCA(n_components=3)

In [4]:
data_pca = pca.fit_transform(bitcoin_df)

In [5]:
data_pca[:5]

array([[-1.79476986e+04, -5.61518209e+01,  2.45745142e+01],
       [-1.79437296e+04, -5.95440539e+01,  1.35642604e+01],
       [-1.79451789e+04, -5.66290337e+01,  1.22716171e+01],
       [-1.79508699e+04, -6.77596411e+01,  1.66330422e+01],
       [-1.79452311e+04, -6.01294107e+01,  8.29945205e+00]])

In [6]:
pca.explained_variance_ratio_

array([9.99967139e-01, 1.82008851e-05, 5.93172708e-06])

In [7]:
total_explained_variance = np.sum(pca.explained_variance_ratio_)
total_explained_variance

0.999991271629944

In [8]:
pca_df = pd.DataFrame(data=data_pca, columns=["PCA1", "PCA2", "PCA3"])

In [9]:
pca_df.head()

,PCA1,PCA2,PCA3
0,-17947.698621,-56.151821,24.574514
1,-17943.729645,-59.544054,13.564260
2,-17945.178910,-56.629034,12.271617
3,-17950.869892,-67.759641,16.633042
4,-17945.231115,-60.129411,8.299452


In [10]:
import alpaca_trade_api as tradeapi
from datetime import datetime, timedelta
import time

# Initialize Alpaca API
api = tradeapi.REST('PKD8BNDR0EC8XAJCLZDY', 'gAgLE5vv1g90cncwc0SQO4K13TeeFwI2gr7caf7p', base_url='https://paper-api.alpaca.markets')  # Use the paper trading API for testing

# Get the current date and time
current_datetime = datetime.now()
#current_date = current_datetime.strftime('%Y-%m-%d')
current_hour = current_datetime.strftime('%H:%M:%S')
start_datetime = current_datetime - timedelta(days=30)

# Slicing by current date minus 30 days in the past to make our data and hour
sliced_df = predictions_df["Predicted"].loc[f'{start_datetime} {current_hour}':]

# if using live hourly data use this
# sliced_df = predictions_df["Predicted"].loc[f'{current_date} {current_hour}':]

while True:
    # Get account information to check available quantity of BTCUSD
    position_info = api.get_position('BTCUSD')
    btcusd_qty_available = float(position_info.qty)
    # sliced_df contains our predictions for the training set
    # You may adjust this based on your specific signal values (e.g., -1 for sell, 1 for buy, 0 for hold)
    for i, signal in enumerate(sliced_df):
        if signal == 1 and btcusd_qty_available > 0:  # Buy signal and available quantity is greater than 0
            # Place a market order to buy
            api.submit_order(
                symbol='BTCUSD',  # Replace with the stock symbol you want to trade
                qty=1,  # Adjust the quantity based on your strategy
                side='buy',
                type='market',
                time_in_force='gtc'
            )
        elif signal == -1 and btcusd_qty_available > 0:  # Sell signal and available quantity is greater than 0
            # Place a market order to sell
            api.submit_order(
                symbol='BTCUSD',
                qty=1,
                side='sell',
                type='market',
                time_in_force='gtc'
        )
        # Sleep for a specific duration (adjust as needed)
        time.sleep(60) # Sleep for 60 seconds
        #time.sleep(3600) # Sleep for 1 hour to mimic dataset hourly data

NameError: name 'predictions_df' is not defined